In [342]:
import numpy as np
import scipy.integrate as integrate
from scipy.signal import fftconvolve
import os,sys
import yaml
import matplotlib.pyplot as plt
import math
import cv2

In [343]:
def sample_coordinate(pixelsize_x = 55e-06,pixelsize_y = 55e-06,fs_size = 2000,ss_size = 2000,focus_x = 1.2e-03,focus_y = 1.0e-03,defocus = 400e-06, det_dist = 14.0e-03, ap_x = 40e-06, ap_y= 40e-06,wl = 7.29e-11):

    xx_span = fs_size * pixelsize_x
    yy_span = ss_size * pixelsize_y 
    x_span_obj =  ap_x / focus_x * defocus 
    y_span_obj =  ap_y / focus_y * defocus
    print(x_span_obj,y_span_obj)
    sintheta_x = (ap_x/2)/np.sqrt((ap_x/2)**2+focus_x**2)
    sintheta_y = (ap_y/2)/np.sqrt((ap_y/2)**2+focus_y**2)
    print(sintheta_x,sintheta_y)
    n_x = int(pow(2, np.ceil(np.log(np.maximum(x_span_obj,ap_x) / wl * 2* sintheta_x)/np.log(2))))
    #n_x = int(np.maximum(x_span_obj,ap_x) / wl * 2* sintheta_x)
    n_y = int(pow(2, np.ceil(np.log(np.maximum(y_span_obj,ap_y) / wl * 2* sintheta_y)/np.log(2))))
    #n_y = int(np.maximum(y_span_obj,ap_y) / wl * 2* sintheta_y)
    n_xx = int(pow(2, np.ceil(np.log(np.maximum(xx_span,x_span_obj) / wl * 2* sintheta_x)/np.log(2))))
    #n_xx = int(np.maximum(xx_span,x_span_obj) / wl * 2* sintheta_x)
    n_yy = int(pow(2, np.ceil(np.log(np.maximum(yy_span,y_span_obj) / wl * 2* sintheta_y)/np.log(2))))
    #n_yy = int(np.maximum(yy_span,y_span_obj) / wl * 2* sintheta_y)
    print(n_x,n_y)
    #nx_arr = np.arange(-n_x//2,n_x//2)
    #ny_arr = np.arange(-n_y//2,n_y//2)
    #ndecx_arr = np.arange(-n_x//2, n_x//2)
    #ndecy_arr = np.arange(-n_y//2, n_y//2)
    
    delta_xlen = ap_x / n_x
    delta_ylen = ap_y / n_y
    delta_xobj = x_span_obj/n_x
    delta_yobj = y_span_obj/n_y
    delta_xdet = xx_span/n_x
    delta_ydet = yy_span/n_y
    
    return delta_xlen,delta_ylen,delta_xobj,delta_yobj,delta_xdet,delta_ydet,n_x,n_y,n_xx,n_yy

In [344]:
def wf_ini1d(corordinate,wavenumber,alpha,focus):
    u0 = np.exp(1.j*wavenumber/2/focus*corordinate**2 + 1e9j*alpha*(corordinate/focus)**3)
    return u0

In [345]:
def wf_ini_noabe_1d(corordinate,wavenumber,alpha,focus):
    u0 = np.exp(1.j*wavenumber/2/focus*corordinate**2)
    return u0

In [346]:
def cov_kernel1d(corordinate,delta_input,delta,wavenumber,distance,wl):
    hn = distance/1.j/np.sqrt(wl)*delta_input*np.exp(-1.j*wavenumber*np.sqrt(((corordinate*delta)**2 + distance**2)))/((corordinate*delta)**2 + distance**2)**0.75
    return hn

In [347]:
def wm(corordinate,n,af = 1):
    wm = np.exp(1.j*np.pi*corordinate**2*af/n)
    return wm

def hm(corordinate,n,af = 1):
    hm = np.exp(-1.j*np.pi*corordinate**2*af/n)
    return hm

In [348]:
with open('configure_simtry.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)

In [349]:
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

ss_size = conf_para['Detector']['ss_size']
fs_size = conf_para['Detector']['fs_size']

pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
wl = conf_para['Source']['wl']
#wl = 7.29e-10
k = 2*np.pi/wl
print(pixelsize_x)

5.5e-05


In [351]:
delta_xlen,delta_ylen,delta_xobj,delta_yobj,delta_xdet,delta_ydet,n_x,n_y,n_xx,n_yy = sample_coordinate(pixelsize_x = pixelsize_x,pixelsize_y = pixelsize_y,fs_size = fs_size,ss_size = ss_size,focus_x = focus_x,focus_y = focus_y,defocus = defocus, det_dist = det_dist, ap_x = ap_x, ap_y= ap_y,wl = wl)
print(n_xx,n_yy)

0.00013333333333333337 0.00016
0.016664352333993337 0.01999600119960014
8192 16384
32768 32768


### Scale factor

In [352]:
deltax = np.maximum(delta_xlen,delta_xobj)
print(deltax)
scale_factor_lenx = delta_xlen/deltax
scale_factor_objx = delta_xobj/deltax
print(scale_factor_lenx,scale_factor_objx)
#M_x = int(n_x * (1 + np.minimum(scale_factor_lenx,scale_factor_objx)))
M_x = int(pow(2, np.ceil(np.log(n_x * (1 + np.minimum(scale_factor_lenx,scale_factor_objx)))/np.log(2))))
print(M_x,(M_x-n_x)//2)

1.627604166666667e-08
0.29999999999999993 1.0
16384 4096


In [356]:
M_x_arr = np.arange(-(M_x),(M_x))
x_len_arr = M_x_arr * delta_xlen
#u0_x = wf_ini_noabe_1d(x_len_arr,wavenumber = k,alpha = alpha_x,focus=focus_x)
u0_x = wf_ini1d(x_len_arr,wavenumber = k,alpha = alpha_x,focus=focus_x)
u0_x[np.abs(x_len_arr)>ap_x/2]=0
#u0_xnew = np.pad(u0_x,((M_x-n_x)//2,(M_x-n_x)//2),'constant',constant_values= (0,0))
%matplotlib widget
plt.plot(x_len_arr,np.abs(u0_x))
#plt.plot(x_len_arr,np.unwrap(np.angle(u0_x)))
print(n_x,u0_x.shape)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8192 (32768,)


In [357]:
x_obj_arr = M_x_arr*delta_xobj
#print(wl)
hn_x = cov_kernel1d(M_x_arr,delta_input = delta_xlen,delta=deltax,wavenumber=k,wl=wl,distance = (focus_x+defocus))
f_M_x_arr = np.fft.fftfreq(M_x*2)
wm_x_len = wm(M_x_arr,n=M_x,af=scale_factor_lenx)
hm_x_len = hm(M_x_arr,n=M_x,af=scale_factor_lenx)
#%matplotlib widget
#plt.plot(wm_x_len)
wm_x_obj = wm(M_x_arr,n=M_x,af=scale_factor_objx)
#print(wm_x_len.shape)
u0_xf = np.fft.fftshift(np.fft.fft(u0_x)/M_x)
hn_xf = np.fft.fftshift(np.fft.fft(hn_x)/M_x)
#print(u0_xf.shape,hn_xf.shape,wm_x_len.shape)
gm_xf = u0_xf*hn_xf*wm_x_len
print(wm_x_obj.shape,gm_xf.shape)
u1_x = wm_x_obj * fftconvolve(gm_xf,hm_x_len,mode='same')

#u1_try = fftconvolve(u0_xf*hn_xf,hm_x_len,mode='same')
%matplotlib widget
#plt.plot(nx_arr)
#plt.plot(x_len_arr)
#plt.plot(x_obj_arr)
px_abs = plt.plot(x_obj_arr,u0_xf)
#px= plt.plot(x_obj_arr,np.-unwrap(np.angle(u1_x)))

(32768,) (32768,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [358]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
#axes[0].plot(np.abs(u1_x[500:2200]))
axes[0].plot(x_obj_arr,np.abs(u1_x))
axes[0].set_title('Lens_x Amplitude', fontsize=10)
axes[0].tick_params(direction='in', length=6, width=2, colors='b',
               grid_color='r', grid_alpha=0.5)
axes[0].xaxis.get_major_formatter().set_powerlimits((0,1))
#axes[0].set_xticklabels(x_obj_arr,fontsize = 8)
#axes[1].plot(np.unwrap(np.imag(u1_x[500:2200])))
axes[1].plot(x_obj_arr,-np.unwrap(np.angle(u1_x)))
axes[1].set_title('Lens_x Phase', fontsize=10)
axes[1].tick_params(direction='in', length=6, width=2, colors='b',
               grid_color='r', grid_alpha=0.5)
axes[1].xaxis.get_major_formatter().set_powerlimits((0,1))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print(u1_x.shape,u1_y.shape)

In [363]:
deltay = np.maximum(delta_ylen,delta_yobj)
scale_factor_leny = delta_ylen/deltay
scale_factor_objy = delta_yobj/deltay
#M_y = int(n_y * (1 + np.minimum(scale_factor_leny,scale_factor_objy)))
M_y = int(pow(2, np.ceil(np.log(n_y * (1 + np.minimum(scale_factor_leny,scale_factor_objy)))/np.log(2))))
print(M_y)

32768


In [364]:
M_y_arr = np.arange(-M_y,M_y)
y_len_arr = M_y_arr * delta_ylen
f_M_y_arr = np.fft.fftfreq(M_y*2)
#u0_y = wf_ini_noabe_1d(y_len_arr,wavenumber = k,alpha = alpha_y,focus=focus_y)
u0_y = wf_ini1d(y_len_arr,wavenumber = k,alpha = alpha_y,focus=focus_y)
u0_y[np.abs(y_len_arr)>ap_y/2]=0
#u0_xnew = np.pad(u0_x,((M_x-n_x)//2,(M_x-n_x)//2),'constant',constant_values= (0,0))
%matplotlib widget
plt.plot(y_len_arr,np.abs(u0_y))
#plt.plot(x_len_arr,np.unwrap(np.angle(u0_x)))
print(n_y,u0_y.shape)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

16384 (65536,)


In [365]:
y_obj_arr = M_y_arr*delta_yobj
#print(wl)
hn_y = cov_kernel1d(M_y_arr,delta_input = delta_ylen,delta=deltay,wavenumber=k,wl=wl,distance = (focus_y+defocus))

wm_y_len = wm(M_y_arr,n=M_y,af=scale_factor_leny)
hm_y_len = hm(M_y_arr,n=M_y,af=scale_factor_leny)
#%matplotlib widget
#plt.plot(wm_x_len)
wm_y_obj = wm(M_y_arr,n=M_y,af=scale_factor_objy)
#print(wm_x_len.shape)
u0_yf = np.fft.fftshift(np.fft.fft(u0_y)/M_y)
hn_yf = np.fft.fftshift(np.fft.fft(hn_y)/M_y)
#print(u0_xf.shape,hn_xf.shape,wm_x_len.shape)
gm_yf = u0_yf*hn_yf*wm_y_len
print(wm_y_obj.shape,gm_yf.shape)
u1_y = wm_y_obj * fftconvolve(gm_yf,hm_y_len,mode='same')

#u1_try = fftconvolve(u0_xf*hn_xf,hm_x_len,mode='same')
%matplotlib widget
#plt.plot(nx_arr)
#plt.plot(x_len_arr)
#plt.plot(x_obj_arr)
plt.plot(y_obj_arr,u0_yf)

(65536,) (65536,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [366]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p1 = axes[0].plot(y_obj_arr,np.abs(u1_y))
axes[0].set_title('Lens_y Amplitude', fontsize=14)
axes[0].tick_params(direction='in', length=6, width=2, colors='b',
               grid_color='r', grid_alpha=0.5)
axes[0].xaxis.get_major_formatter().set_powerlimits((0,1))

p2 = axes[1].plot(y_obj_arr,-np.unwrap(np.angle(u1_y)))
axes[1].set_title('Lens_y Phase', fontsize=14)
axes[1].tick_params(direction='in', length=6, width=2, colors='b',
               grid_color='r', grid_alpha=0.5)
axes[1].xaxis.get_major_formatter().set_powerlimits((0,1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [367]:
print(scale_factor_lenx,scale_factor_objx)
print(scale_factor_leny,scale_factor_objy)
print(u1_x.shape,u1_y.shape)

0.29999999999999993 1.0
0.25 1.0
(32768,) (65536,)


In [299]:
print(n_xx)

4096


In [370]:
u1_ytry = u1_y
u1_x = np.reshape(u1_x,(1,len(u1_x)))
u1_y = np.reshape(u1_y,(len(u1_y),1))
#print(u1_y.shape)

In [371]:
u1 = u1_x * u1_y
print(u1.shape)

(65536, 32768)


In [369]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(x_obj_arr)
x_max = np.max(x_obj_arr)
y_min = np.min(y_obj_arr)
y_max = np.max(y_obj_arr)
p1 = axes[0].imshow(np.abs(u1),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Lens Amplitude at defocus plane', fontsize=10)


p2 = axes[1].imshow(-np.unwrap(np.angle(u1)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Lens Phase at defocus plane', fontsize=10)

plt.colorbar(p1,ax = axes[0])
plt.colorbar(p2,ax = axes[1])

plt.savefig('Lens_1D.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [303]:
%matplotlib widget
plt.plot(-np.unwrap(np.angle(u1[:,1])))
#plt.plot(-np.unwrap(np.angle(u1[10000])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [304]:
u1_trans = np.pad(u1,(np.abs(n_yy - u1.shape[0])//2,np.abs(n_xx-u1.shape[1])//2),'constant',constant_values= (0,0))
print(u1_trans.shape)

(4096, 4096)


In [305]:
img = cv2.imread('arrow4.png',0)
#%matplotlib widget
#plt.imshow(img)
#img2 = cv2.resize(img,(2*M_x,2*M_y))
#img2 = cv2.resize(img,(u1_trans.shape[1],u1_trans.shape[0]))
img2 = cv2.resize(img,(u1.shape[1],u1.shape[0]))
%matplotlib widget
#plt.imshow(img2)

In [306]:
print(img2.shape)

(4096, 4096)


In [307]:
#u_input = u1_trans * img2
u_input = u1 * img2

In [308]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p3 = axes[0].imshow(np.abs(u_input),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude with Arrow', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(u_input)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase with Arrow', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('defocus_2D.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### construct the 2D object


### The propagate image on the detector plane

In [309]:
def cov_kernel2d(corordinatex,corordinatey,delta_inputx,delta_inputy,deltax,deltay,wavenumber,distance,wl=7.29e-11):
    hn = distance/1.j/np.sqrt(wl)*delta_inputx*delta_inputy*np.exp(-1.j*wavenumber*np.sqrt((corordinatex*deltax)**2 +(corordinatey*deltay)**2+distance**2))/((corordinatex*deltax)**2 +(corordinatey*deltay)**2+distance**2)**0.75
    return hn

In [310]:
deltax = np.maximum(delta_xobj,delta_xdet)
deltay = np.maximum(delta_yobj,delta_ydet)
scale_factor_objx = delta_xobj/deltax
scale_factor_objy = delta_yobj/deltay
scale_factor_detx = delta_xdet/deltax
scale_factor_dety = delta_ydet/deltay
print(scale_factor_objx,scale_factor_detx)
print(scale_factor_objy,scale_factor_dety)
M_x_new = int(pow(2, np.ceil(np.log(n_xx * (1 + np.minimum(scale_factor_objx,scale_factor_detx)))/np.log(2))))
M_y_new = int(pow(2, np.ceil(np.log(n_yy * (1 + np.minimum(scale_factor_objy,scale_factor_dety)))/np.log(2))))
#M_x_new = int(n_xx * (1 + np.minimum(scale_factor_objx,scale_factor_detx)))
#M_y_new = int(n_yy * (1 + np.minimum(scale_factor_objy,scale_factor_dety)))
print(M_x_new,M_y_new)

0.24242424242424246 1.0
0.2909090909090909 1.0
8192 8192


In [322]:
u1_new = np.pad(u_input,((M_y_new//4),(M_x_new//4)),'constant',constant_values= (0,0))
print(u_input.shape)
print(u1_new.shape)

(4096, 4096)
(8192, 8192)


In [323]:
%matplotlib widget
plt.imshow(np.abs(u1_new))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [324]:
M_xnew_arr = np.arange(-u1_new.shape[1]//2,u1_new.shape[1]//2)
M_ynew_arr = np.arange(-u1_new.shape[0]//2,u1_new.shape[0]//2)
x_len_arr_new = np.reshape(M_xnew_arr,(1,len(M_xnew_arr)))
y_len_arr_new = np.reshape(M_ynew_arr,(len(M_ynew_arr),1))
#len_arr = x_len_arr_new*deltax*y_len_arr_new*deltay
print(x_len_arr_new.shape,y_len_arr_new.shape)

(1, 8192) (8192, 1)


In [325]:
def wm2d(corordinatex,corordinatey,nx,ny,afx = 1,afy = 1):
    wm2d = np.exp(1.j*np.pi/nx*corordinatex**2*afx)*np.exp(1.j*np.pi/ny*corordinatey**2*afy)
    return wm2d

def hm2d(corordinatex,corordinatey,nx,ny,afx = 1,afy = 1):
    hm2d = np.exp(-1.j*np.pi/nx*corordinatex**2*afx)*np.exp(-1.j*np.pi/ny*corordinatey**2*afy)
    return hm2d

In [326]:
M_y_2d,M_x_2d = u1_new.shape
print(M_x_2d)

8192


In [327]:
hn_det = cov_kernel2d(x_len_arr_new,y_len_arr_new,delta_xobj,delta_yobj,deltax,deltay,wavenumber=k,wl=wl,distance = det_dist)
#f_x_lenarr_new = np.reshape(np.fft.fftfreq(M_x_2d),(1,len(M_xnew_arr)))
#f_y_lenarr_new = np.reshape(np.fft.fftfreq(M_y_2d),(len(M_ynew_arr),1))
wm_obj = wm2d(x_len_arr_new,y_len_arr_new,nx = M_x_2d,ny = M_y_2d,afx = scale_factor_objx,afy = scale_factor_objy)
hn_obj = hm2d(x_len_arr_new,y_len_arr_new,nx = M_x_2d,ny = M_y_2d,afx = scale_factor_objx,afy = scale_factor_objy)
wm_det = wm2d(x_len_arr_new,y_len_arr_new,nx = M_x_2d,ny = M_y_2d,afx = scale_factor_detx,afy = scale_factor_dety)

In [328]:
#u1ff = np.fft.fft2(u1_new)
#u1f = np.fft.fftshift(np.fft.fft2(u1_new))
u1f = np.fft.fftshift(np.fft.fft2(u1_new)/len(M_xnew_arr)/len(M_ynew_arr))
hn_detf = np.fft.fftshift(np.fft.fft2(hn_det)/len(M_xnew_arr)/len(M_ynew_arr))
#hn_detf = np.fft.fftshift(np.fft.fft2(hn_det))
gm_det = u1f*hn_detf*wm_obj
#print(wm_obj.shape)

In [341]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p7 = axes[0].imshow(np.abs(hn_det))
axes[0].set_title('Kernel Amplitude', fontsize=14)
p8 = axes[1].imshow(np.unwrap(np.angle(hn_det)))
axes[1].set_title('Kernel Phase', fontsize=14)
plt.colorbar(p7,ax = axes[0])
plt.colorbar(p8,ax = axes[1])
plt.savefig('Kernel2D.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [330]:
%matplotlib widget
p1 = plt.imshow(np.abs(gm_det))
plt.colorbar(p1)
plt.savefig('Kernel_2d_%s.png' % (det_dist))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [333]:
print(gm_det.shape,hn_obj.shape,wm_det.shape)
#u1_y = wm_obj * fftconvolve(gm_det,hn_obj,mode='same')
u1_y = wm_det * fftconvolve(gm_det,hn_obj,mode='same')

(8192, 8192) (8192, 8192) (8192, 8192)


In [334]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
p1 = axes[0].imshow(np.abs(u1_y))
axes[0].set_title('Amplitude on detector plane', fontsize=14)
p2 = axes[1].imshow(-np.unwrap(np.angle(u1_y)))
axes[1].set_title('Phase on detector plane', fontsize=14)
plt.colorbar(p1,ax = axes[0])
plt.colorbar(p2,ax = axes[1])
plt.savefig('Lens_propagate2_%s.png'%(det_dist))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …